In [1]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////home/marialejandramantillab/dmeyf.db

In [3]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('~/buckets/b1/datasets/competencia_02_crudo.csv.gz')

,Success


In [4]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_02
), clientes as (
    select distinct numero_de_cliente from competencia_02
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , CASE
            WHEN mes_1 IS NOT NULL
                 AND mes_2 IS NOT NULL
            THEN 'CONTINUA'
            WHEN mes_1 IS NULL
            THEN 'BAJA+1'
            WHEN mes_1 IS NOT NULL
                 AND mes_2 IS NULL
            THEN 'BAJA+2'
        END as clase_ternaria 
    from todo t
    left join competencia_02 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria where mes_0 = 1

,Success


In [5]:
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)

,Success


In [6]:
%%sql
update competencia_02
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_02.numero_de_cliente = targets.numero_de_cliente and competencia_02.foto_mes = targets.foto_mes;

,Success


In [7]:
%%sql
copy competencia_02 to '~/buckets/b1/datasets/competencia_02.csv.gz' (FORMAT CSV, HEADER)

,Success
